In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.stats import norm

def black_scholes(S, K, T, r, sigma, option_type="call"):
    """Black-Scholes formula to calculate option price and time value"""
    if T <= 0:  # Ensure time doesn't go to zero to avoid division by zero
        return max(0, S - K) if option_type == "call" else max(0, K - S)
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == "call":
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return option_price

def option_profit_table(symbol, 
                        current_price, 
                        option_type, 
                        position, 
                        date_range, 
                        strike_price, 
                        premium, 
                        contract_size=100, 
                        price_step=1, 
                        r=0.01,  # interest rate
                        sigma=0.3):  # implied volatility
    
    # Generate a range of stock prices
    stock_price_range = np.arange(int(current_price * 0.95), int(current_price * 1.05) + 1, price_step)

    # Initialize the profit matrix
    profit_matrix = []

    # Generate a full range of dates between start and end date
    date_range = pd.date_range(start=date_range[0], end=date_range[1])

    # Convert date_range into time to expiry
    expiration_date = date_range[-1]
    time_to_expiration = (expiration_date - date_range).days / 365.0

    # Calculate profit at expiry for each stock price and expiration date
    for stock_price in stock_price_range:
        daily_profits = []
        for T in time_to_expiration:
            # Calculate the time value (using Black-Scholes model)
            time_value = black_scholes(stock_price, strike_price, T, r, sigma, option_type)

            if option_type == 'call':
                if position == 'buy':
                    value_at_expiry = max(stock_price - strike_price, 0)
                    profit = (value_at_expiry - premium) * contract_size if T == 0 else (value_at_expiry - premium + time_value) * contract_size
                elif position == 'write':
                    value_at_expiry = max(stock_price - strike_price, 0)
                    profit = (premium * contract_size) - (value_at_expiry * contract_size) if T == 0 else (premium * contract_size) - (value_at_expiry + time_value) * contract_size

            elif option_type == 'put':
                if position == 'buy':
                    value_at_expiry = max(strike_price - stock_price, 0)
                    profit = (value_at_expiry - premium) * contract_size if T == 0 else (value_at_expiry - premium + time_value) * contract_size
                elif position == 'write':
                    value_at_expiry = max(strike_price - stock_price, 0)
                    max_loss = (strike_price * contract_size) - (premium * contract_size)  # Max loss in case stock goes to 0
                    profit = (premium * contract_size) - (value_at_expiry * contract_size) if T == 0 else (premium * contract_size) - (value_at_expiry + time_value) * contract_size
                    profit = max(profit, -max_loss)

            daily_profits.append(profit)
        profit_matrix.append(daily_profits)

    # Create a Pandas DataFrame with the expiration dates as columns and stock prices as rows
    profit_df = pd.DataFrame(profit_matrix, columns=[f"{date.date()}" for date in date_range], index=[f"${price:.2f}" for price in stock_price_range])

    # Add a summary section for estimated returns, breakeven, and risk
    if option_type == 'call':
        breakeven_price = strike_price + premium
    elif option_type == 'put':
        breakeven_price = strike_price - premium

    summary = {
        'Symbol': symbol,
        'Current Price': current_price,
        'Strike Price': strike_price,
        'Premium Paid' if position == 'buy' else 'Premium Received': premium,
        'Breakeven Price': breakeven_price,
    }

    # Update for call or put options
    if option_type == 'call':
        summary.update({
            'Max Profit': 'infinite' if position == 'buy' else 'Limited to premium received',
            'Max Risk': premium * contract_size if position == 'buy' else 'Unlimited Risk',
        })
    elif option_type == 'put':
        if position == 'buy':
            summary.update({
                'Max Profit': f'Limited to {strike_price - premium} per contract',
                'Max Risk': premium * contract_size,
            })
        elif position == 'write':
            # For written puts, max profit is the premium received, max risk is the strike price minus premium
            max_risk = (strike_price * contract_size) - (premium * contract_size)
            summary.update({
                'Max Profit': premium * contract_size,  # Premium received
                'Max Risk': max_risk,  # Risk if stock drops to 0
            })

    summary['Breakeven at Expiry'] = breakeven_price

    return profit_df, summary

# Example usage
symbol = "SPY"
current_price = 538.91
option_type = "call"  # can be "call" or "put"
position = "write"  # can be "buy" or "write"
date_range = ["2024-09-06", "2024-09-19"]  # Date range including expiration date
strike_price = 520
premium = 2.50  # Premium paid, aka. price per option
contract_size = 100

# Call the function
profit_df, summary = option_profit_table(symbol, current_price, option_type, position, date_range, strike_price, premium, contract_size, price_step=2)

# Display the summary
print("Summary Section:")
for k, v in summary.items():
    print(f"{k}: {v}")

# Plot the profit/loss table using Plotly
fig = go.Figure()

# Add the heatmap
fig.add_trace(go.Heatmap(
    z=profit_df.values,
    x=profit_df.columns,
    y=profit_df.index,
    colorscale='RdYlGn',
    hoverongaps=False,
    colorbar=dict(title='Profit / Loss ($)')
))

# Add text annotations for the values in each cell
for i, row in enumerate(profit_df.values):
    for j, val in enumerate(row):
        fig.add_annotation(
            go.layout.Annotation(
                text=f'{val:.0f}',  # Formatting value as integer
                x=profit_df.columns[j],
                y=profit_df.index[i],
                showarrow=False,
                font=dict(color="black" if -100 < val < 100 else "white"),
            )
        )

# Customize layout
fig.update_layout(
    title=f'Options Profit and Loss Table for {symbol} ({option_type.title()} Option)',
    xaxis_title="Date",
    yaxis_title="Stock Price",
    xaxis=dict(tickmode='array', tickvals=profit_df.columns),
    yaxis=dict(tickmode='array', tickvals=profit_df.index),
    font=dict(family="Arial", size=12),
    height=800,
    width=900,
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(255,255,255,1)',
    margin=dict(l=100, r=100, t=100, b=100),
)

# Show the plot
fig.show()
